In [1]:
import pandas as pd
import json
from tqdm import tqdm

In [2]:
# game classification values extraction functions

# get all versions of games availiable in data
def get_game_versions(data=[]):
    game_versions = []
    for game_version in data:
        game_versions.append(game_version['info']['game_version'])
    return set(game_versions)

# get all game types availiable in data
def get_game_types(data=[]):
    game_types = []
    for game_type in data:
        game_types.append(game_type['info']['tft_game_type'])
    return set(game_types)

# get all sets availiable in data
def get_set(data=[]):
    sets = []
    for set_name in data:
        try:
            sets.append(set_name['info']['tft_set_core_name'])
        except:
            pass
    return set(sets)

# get all queue types availiable in data
def get_queue(data=[]):
    queues = []
    for queue in data:
        queues.append(queue['info']['queue_id'])
    return set(queues)

In [8]:
# extract matches of one version of game
def get_matches_by_game_version(data=[], game_ver=''):    
    one_version_matches = []
    for match in data:
        if match['info']['game_version'].split(' ')[-1] == game_ver:
            one_version_matches.append(match)
    return one_version_matches

# extract matches of one game type
def get_matches_by_game_type(data=[], game_type=''):
    matches_by_gametype = []
    for match in data:
        if match['info']['tft_game_type'] == game_type:
            matches_by_gametype.append(match)
    return matches_by_gametype

# extract matches of one set
def get_matches_by_set(data=[], game_set=[]):    
    one_set_matches = []
    for match in tqdm(data):
        try:
            if match['info']['tft_set_core_name'] in game_set:
                one_set_matches.append(match)
        except:
            pass
    return one_set_matches

In [10]:
# get some augments stats functions
def get_augments_placement(data=[]):
    augments = []
    for j in tqdm(data):
        for i in range(len(j['info']['participants'])):
            d = {'augments':j['info']['participants'][i]['augments'], 
                 'placement':j['info']['participants'][i]['placement'], 
                 'patch':j['info']['game_version'],
                 'set':j['info']['tft_set_core_name'],
                 'region':j['region']}
            augments.append(d)
    return augments

def get_augments_frequency(data=pd.DataFrame()):
    return data.groupby(['augments', 'patch', 'set', 'region'], as_index=False) \
                .agg({'placement':'count'}) \
                .sort_values(by='placement', ascending=False) \
                .rename(columns={'placement':'frequency'})

def get_augments_placement_mean(data=pd.DataFrame()):
    return data.groupby(['augments', 'patch', 'set', 'region'], as_index=False) \
                .agg({'placement':'mean'}) \
                .sort_values(by='placement') \
                .round(2)

In [11]:
# get characters bound with items and their placement in game
def get_char_item_placement(data=[]):
    char_items_placement = []
    for game in tqdm(data):
        for i in range(len(game['info']['participants'])):
            for j in range(len(game['info']['participants'][i]['units'])):
                d = {'character_id':game['info']['participants'][i]['units'][j]['character_id'],
                     'itemNames':   game['info']['participants'][i]['units'][j]['itemNames'],
                     'rarity':      game['info']['participants'][i]['units'][j]['rarity'],
                     'tier':        game['info']['participants'][i]['units'][j]['tier'],
                     'placement':   game['info']['participants'][i]['placement'],
                     'patch':       game['info']['game_version'],
                     'set':         game['info']['tft_set_core_name'],
                     'region':      game['region']}
                char_items_placement.append(d)
    return char_items_placement

# get only char placements
def get_char_placement(data=[]):
    char_placement = []
    for game in tqdm(data):
        for i in range(len(game['info']['participants'])):
            for j in range(len(game['info']['participants'][i]['units'])):
                d = {'character_id':game['info']['participants'][i]['units'][j]['character_id'],
                     'rarity':      game['info']['participants'][i]['units'][j]['rarity'],
                     'tier':        game['info']['participants'][i]['units'][j]['tier'],
                     'placement':   game['info']['participants'][i]['placement'],
                     'patch':       game['info']['game_version'],
                     'set':         game['info']['tft_set_core_name'],
                     'region':      game['region']}
                char_placement.append(d)
    return char_placement

In [6]:
# utility functions

# patch naming can be frustrating. Stripping function to simple 'xx.xx' type
def simplify_patch_string(data=pd.DataFrame):
    data.patch = data.patch.apply(lambda x: x.split('Releases/')[-1][:-1])
    return data

# func to make augment names more readable
def aug_name_split(data=pd.DataFrame):
    return data.apply(lambda data: data.split('_Augment_')[-1])

In [7]:
with open('path to parsed jsons', 'r') as r:
    df = json.load(r)

In [66]:
len(df)

63795

In [13]:
get_game_types(df)

{'pairs', 'standard', 'turbo', 'tutorial'}

In [14]:
get_game_versions(df)

{'Version 12.23.483.5208 (Dec 05 2022/14:03:09) [PUBLIC] <Releases/12.23>',
 'Version 13.1.487.9641 (Jan 06 2023/15:09:01) [PUBLIC] <Releases/13.1>',
 'Version 13.1.489.0970 (Jan 12 2023/13:44:40) [PUBLIC] <Releases/13.1>',
 'Version 13.1.489.3737 (Jan 13 2023/15:43:46) [PUBLIC] <Releases/13.1>',
 'Version 13.3.491.0366 (Feb 06 2023/15:18:04) [PUBLIC] <Releases/13.3>',
 'Version 13.3.491.6222 (Feb 09 2023/14:51:50) [PUBLIC] <Releases/13.3>',
 'Version 13.4.493.7613 (Feb 21 2023/16:23:09) [PUBLIC] <Releases/13.4>',
 'Version 13.4.494.1549 (Feb 23 2023/13:51:24) [PUBLIC] <Releases/13.4>',
 'Version 13.5.495.8836 (Mar 03 2023/18:17:19) [PUBLIC] <Releases/13.5>',
 'Version 13.6.499.0913 (Mar 20 2023/16:29:01) [PUBLIC] <Releases/13.6>',
 'Version 13.6.499.7165 (Mar 23 2023/14:00:55) [PUBLIC] <Releases/13.6>',
 'Version 13.6.499.7758 (Mar 23 2023/18:53:26) [PUBLIC] <Releases/13.6>',
 'Version 13.7.501.1316 (Mar 30 2023/15:09:16) [PUBLIC] <Releases/13.7>',
 'Version 13.7.502.2251 (Apr 05 2023

In [15]:
get_queue(df)

{1090, 1100, 1110, 1130, 1160}

In [16]:
get_set(df)

{'TFTSet8', 'TFTSet8_2', 'TFTTutorial'}

In [17]:
# extract set 8.5, standard game type matches from matches pile
matches_sorted = get_matches_by_game_type(get_matches_by_set(data=df, game_set=['TFTSet8_2']), game_type='standard')

100%|███████████████████████████████████████████████████████████████████████| 63795/63795 [00:00<00:00, 1315055.33it/s]


In [18]:
len(matches_sorted)

60008

In [29]:
# df for position(1, 2, 3) divided augments
augs_placement_pos_div = simplify_patch_string(pd.DataFrame(get_augments_placement(data=matches_sorted)))

100%|█████████████████████████████████████████████████████████████████████████| 60008/60008 [00:03<00:00, 17711.40it/s]


In [30]:
# transforming df to get placements of augments sequences
augs_placement_pos_div['first_aug']  = augs_placement_pos_div.augments.str[0]
augs_placement_pos_div['second_aug'] = augs_placement_pos_div.augments.str[1]
augs_placement_pos_div['third_aug']  = augs_placement_pos_div.augments.str[2]

augs_placement_pos_div.third_aug.fillna('None', inplace=True)
augs_placement_pos_div = augs_placement_pos_div.dropna()

augs_placement_pos_div[['first_aug', 'second_aug', 'third_aug']] = augs_placement_pos_div[['first_aug', 'second_aug', 'third_aug']].apply(aug_name_split)
augs_placement_pos_div.drop(columns=['augments'], inplace=True)

In [42]:
augs_placement_pos_div

,placement,patch,set,region,first_aug,second_aug,third_aug
0,5,13.9,TFTSet8_2,BR1,ScopedWeapons1,JaxASCarry,SecondWind1
1,7,13.9,TFTSet8_2,BR1,Distancing2,NilahSupport,PandorasItems
2,8,13.9,TFTSet8_2,BR1,PortableForge,RellCarry,FuturePeepers
3,2,13.9,TFTSet8_2,BR1,TargetDummies,ShenSupport,SecondWind1
4,1,13.9,TFTSet8_2,BR1,JeweledLotus,CamilleSupport,HackerTrait
...,...,...,...,...,...,...,...
480059,5,13.8,TFTSet8_2,TR1,InfiniteamEmblem,PortableForge,TwistedFateSupport
480060,4,13.8,TFTSet8_2,TR1,TrueTwos,RichGetRicherPlus,NeekoSupport
480061,8,13.8,TFTSet8_2,TR1,RichGetRicher,TradeSectorPlus,GarenSupport
480062,6,13.8,TFTSet8_2,TR1,Distancing2,ComponentGrabBag,TwistedFateCarry


In [39]:
# saving
augs_placement_pos_div.to_csv('path', sep=',')

___

In [43]:
# simple characters placement
chars_placement = simplify_patch_string(data=pd.DataFrame(get_char_placement(data=matches_sorted)))

100%|█████████████████████████████████████████████████████████████████████████| 60008/60008 [00:04<00:00, 12107.10it/s]


In [44]:
chars_placement

,character_id,rarity,tier,placement,patch,set,region
0,TFT8_Blitzcrank,0,2,5,13.9,TFTSet8_2,BR1
1,TFT8_Vi,1,2,5,13.9,TFTSet8_2,BR1
2,TFT8_Pyke,1,1,5,13.9,TFTSet8_2,BR1
3,TFT8_Riven,2,3,5,13.9,TFTSet8_2,BR1
4,TFT8_Jax,2,3,5,13.9,TFTSet8_2,BR1
...,...,...,...,...,...,...,...
3902033,TFT8_Ekko,4,2,1,13.8,TFTSet8_2,TR1
3902034,TFT8_Viego,4,2,1,13.8,TFTSet8_2,TR1
3902035,TFT8_TwistedFate,4,2,1,13.8,TFTSet8_2,TR1
3902036,TFT8_Neeko,4,2,1,13.8,TFTSet8_2,TR1


___

In [45]:
# creating char-item dependent placement df
chars_items_placement = simplify_patch_string(data=pd.DataFrame(get_char_item_placement(data=matches_sorted)).explode('itemNames'))

100%|██████████████████████████████████████████████████████████████████████████| 60008/60008 [00:09<00:00, 6272.55it/s]


In [47]:
# cleaning and making readable naming for chars and items
chars_items_placement = chars_items_placement.dropna()

chars_items_placement.character_id = chars_items_placement.character_id.apply(lambda x: x.split('_')[-1])

chars_items_placement.itemNames = chars_items_placement.itemNames.apply(lambda x: x.split('_Item_')[-1])

In [78]:
chars_items_placement

,character_id,itemNames,rarity,tier,placement,patch,set,region
3,Riven,RedBuff,2,3,5,13.9,TFTSet8_2,BR1
3,Riven,TitansResolve,2,3,5,13.9,TFTSet8_2,BR1
3,Riven,FrozenHeart,2,3,5,13.9,TFTSet8_2,BR1
4,Jax,JeweledGauntlet,2,3,5,13.9,TFTSet8_2,BR1
4,Jax,Bloodthirster,2,3,5,13.9,TFTSet8_2,BR1
...,...,...,...,...,...,...,...,...
3902036,Neeko,HextechGunblade,4,2,1,13.8,TFTSet8_2,TR1
3902036,Neeko,SpearOfShojin,4,2,1,13.8,TFTSet8_2,TR1
3902036,Neeko,JeweledGauntlet,4,2,1,13.8,TFTSet8_2,TR1
3902037,Janna,SpearOfShojin,6,2,1,13.8,TFTSet8_2,TR1


In [81]:
# saving 
chars_items_placement.to_csv('path', sep=',')

___

In [6]:
# creating lists of different types of items
item_list = list(chars_items_placement.itemNames.unique())

radiant_items = [i for i in item_list if 'Radiant' in i]

emblem_item   = [i for i in item_list if 'EmblemItem' in i]

genae_items   = [i for i in item_list if 'GenAE' in i]

ornn_items    = [i for i in item_list if 'Ornn' in i]

# items to exclude
exclusive_items = emblem_item + genae_items + ornn_items + radiant_items

component_items = ['TFT_Item_BFSword', 'TFT_Item_RecurveBow', 'TFT_Item_ChainVest', 'TFT_Item_NegatronCloak', 'TFT_Item_NeedlesslyLargeRod', 'TFT_Item_TearOfTheGoddess', 'TFT_Item_GiantsBelt', 'TFT_Item_SparringGloves', 'TFT_Item_Spatula']

basic_items = ['TFT_Item_ArchangelsStaff', 'TFT_Item_Bloodthirster', 'TFT_Item_BrambleVest', 'TFT_Item_Chalice', 'TFT_Item_Deathblade', 'TFT_Item_DragonsClaw', 'TFT_Item_ForceOfNature', 'TFT_Item_GargoyleStoneplate', 'TFT_Item_GuinsoosRageblade', 'TFT_Item_HextechGunblade', 'TFT_Item_InfinityEdge', 'TFT_Item_IonicSpark', 'TFT_Item_JeweledGauntlet', 'TFT_Item_LastWhisper', 'TFT_Item_LocketOfTheIronSolari', 'TFT_Item_Morellonomicon', 'TFT_Item_Quicksilver', 'TFT_Item_RabadonsDeathcap', 'TFT_Item_RapidFireCannon', 'TFT_Item_Redemption', 'TFT_Item_RunaansHurricane', 'TFT_Item_Shroud', 'TFT_Item_SpearOfShojin', 'TFT_Item_StatikkShiv', 'TFT_Item_ThiefsGloves',  'TFT_Item_TitansResolve',  'TFT_Item_WarmogsArmor', 'TFT_Item_ZekesHerald', 'TFT_Item_Zephyr']

In [ ]:
# division by a type of item holded
chars_items_plmnt_default_items = chars_items_placement.query('itemNames not in @exclusive_items')

chars_items_plmnt_radiant_items = chars_items_placement.query('itemNames in @radiant_items')

chars_items_plmnt_ornn_items = chars_items_placement.query('itemNames in @ornn_items')
chars_items_plmnt_ornn_items.itemNames = chars_items_plmnt_ornn_items.itemNames.apply(lambda x: x.split('Ornn')[-1])

In [8]:
# saving 
chars_items_plmnt_default_items.to_csv('path', sep=',')

chars_items_plmnt_radiant_items.to_csv('path', sep=',')

chars_items_plmnt_ornn_items.to_csv('path', sep=',')